<font size = 6em color = "stealblue"> **Fallbeispiel "Play Tennis"** 🎾 </font><font size = 4em>| Klassifikation mit "Decision Tree"</font>
********
Im Fallbeispiel geht es um eine Klassifikation. Anhand von Wetterdaten soll vorhergesagt werden, ob an einem bestimmten Tag, an dem die relevanten Wetterbedingungen bekannt sind, Tennis gespielt oder nicht Tennis gespielt wird. Die Wetterdaten sind die Feautures bzw. Input-Merkmale und das Ziel-Merkmal bzw. Target die Entscheidung, ob Tennis gespielt oder nicht gespielt wird. Das Target ist somit binär. Die Input-Merkmale sind allesamt kategorial. 
********

# Data Understanding

## Einlesen der Daten aus einer Excel-Datei

In [8]:
import pandas as pd
from pandas import ExcelFile 
import numpy as np
import matplotlib.pyplot as mpl

*Aufgabe*: Lesen Sie den Datenbestand als csv ein: 
- in "data_train": trainingsdaten_playtennis.csv
- in "data_test": testdaten_playtennis.csv

Zum Laden von Daten besitzt pandas für jedes Format eine eigene Funktion. Diese sind alle unter dem folgenden dokumentiert: https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

Außerdem gibt es ein allgemeines Tutorial zum Laden von Daten, aus dem man auch etwas Code kopieren und ausprobieren kann: https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/02_read_write.html


*Aufgabe*: Geben Sie die ersten 10 Zeilen des Trainingsdatensatzes aus.


## Entfernen von irrelevanten Spalten

*Aufgabe*: Entfernen Sie dafür die Spalte "Day". 
Die Spalte dient im Datensatz als ID ("Identifier") und hat daher keinen analytischen Nutzen.

## Erste Inspektion des Dataframe

In [3]:
data_train.shape # gibt die Anzahl der Zeilen/ Instanzen (14) und Merkmale (5) aus. "Day" wurde ja entfernt

(14, 5)

In [4]:
# gibt die Datentypen aus. Es gibt keine metrisch skalierten Merkmale im Trainingsdatenbestand
# es gibt keine missing values
data_train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Outlook      14 non-null     object
 1   Temperature  14 non-null     object
 2   Humidity     14 non-null     object
 3   Wind         14 non-null     object
 4   PlayTennis   14 non-null     object
dtypes: object(5)
memory usage: 344.0+ bytes


# Data Preperation & Modelling (Decision tree)

## Aufteilen in Features und Target
Für die Klassifikation ist es nötig, die im Datenbestand enthaltenen Merkmale nach Features (Input-Merkmale) und Target (Zielvariable/Output) zu trennen. 
Sklearn erhält beim Training eines Estimators normalerweise immer 2 Eingaben: Den Zielwert sowie die gesammelten Merkmale, beide in der gleichen Sortierung.
Also müssen wir unsere Daten trennen und dann in die Variablen X (Merkmale) sowie y (Zielwert) speichern.

*Aufgabe*: Teilen Sie die Trainingsdaten in 
- x_train mit den Input-Merkmalen
- y_train mit der Zielvariable

*Aufgabe*: Wiederholen Sie dasselbe mit den Testdaten.
Hinweis: Bevor Sie aber die Testdaten splitten, speichern sie die Testdatei (für das Testen später) nochmal als "data_test_orig" ab.

## Transformation der nominalen bzw. kategorialen Variablen


Für die Erstellung des Entscheidungsbaums müssen die nominalen bzw. kategorialen Variablen transformiert werden. 
D.h. die Merkmale werden in (Ganz-)Zahlen umgewandelt.
Dafür gibt es zwei Möglichkeiten: Recodierung und One Hot Encoding.
Im Folgenden werden wir beide Möglichkeien ausprobieren und euch die Vor- und Nachteile der jeweiligen Möglichkeiten euch näher bringen.

Einen guten Überblick bietet das "Getting Started"-Tutorial zu scikit-learn: https://scikit-learn.org/stable/getting_started.html


### 1. Möglichkeit: Recodierung

Recodierung ist die Umcodierung von nominal skaliererten Merkmalen in willkürliche Zahlen. 
Beispiel: 
Nehmen wir das Merkmal "Outlook" mit den drei Ausprägungen "overcast", "sunny" und "rain". 
Wenn wir diese Ausprägungen umcodieren, so würde z.B. "overcast" mit einer 2 ersetzt werden, "sunny" mit einer 2 und "rain" mit einer 5. 

D.h. den Ausprägungen wird eine zufällige Zahl zugeordnet ohne diesen eine Bedeutung zuzuordnen. 

Deshalb ist es hier wichtig zu beachten, dass der im Anschluss angewendete Lernalgorithmus die (numerisch) transformierten Werte nicht für Berechnungen (wie z.B. Berechnung von metrischen Abständen) nutzt. 
Ansonsten kann es, dadurch das es willkürlich gewählte Zahlen sind, welche keine nähere Bedeutung wie z.B. eine Rangfolge haben, zu Verzerrungen in den Ergebnissen kommen. 

Für den Entscheidungsbaum verwenden wir hier einen ID3-Algorithmus, welcher nur die Häufigkeiten der Merkmalsausprägungen verwendet und dadurch wäre eine Transformation in beliebige Zahlen unkritisch. 

*Aufgabe*: Führen Sie eine Recodierung der Spalten "Outlook","Wind","Temperature","Humidity" mit der Funktion `LabelEncoder()` aus dem Package scikit-learn durch.

In [ ]:
from sklearn.preprocessing import LabelEncoder


### 2. Möglichkeit: One Hot Encoding

Beim One Hot Encoding wird für jede Ausprägung eines Merkmals eine extra Spalte (neues Merkmal) gebildet. 
Zum Beispiel beim Merkmal "Outlook" mit den Ausprägungen "overcast", "sunny" und "rain" werden folgende neue Spalten gebildet: 
- "outlook_overcast", "outlook_sunny" und "outlook_rain"
- das bisherige Merkmal "Outlook" wird dafür entfernt
- mit 0 und 1 wird dann kenntlich gemacht, welches der drei neuen Merkmale (ehemals Ausprägungen eines Merkmals) bei einer Instanz zutreffen. 

*Aufgabe*: Führen Sie eine Transformation mittels One Hot Encoding durch. Nutzen sie dafür den Befehl "get_dummies" aus dem Python-Package pandas.

### Fazit

One Hot Encoder ist in der Anwendung unproblematischer als die Recodierung, da keine Überprüfung des nachfolgenden Algorithmus hinsichtlich der fehlenden Variablenbedeutung statt finden muss. Jedoch ist zu berücksichtigen, dass durch die Transformation mittels One Hot Encoding die Performance bei der Durchführung des Programmiercodes negativ beeinflusst werden kann und die Unübersichtlichkeit des Datenbestandes aufgrund der dazugekommenen Merkmalsausprägungen steigt. 

Im Folgenden werden wir deshalb die "einfache" Recodierung nutzen, um die Anzahl der Spalten überschaubarer zu halten. 
One Hot Encoder bleibt und ist aber eine wichtige Basis-Transformation, weshalb wir hier die Funktionsweise und den dafür notwendigen Code beheandelt haben. 

## Umwandlung in numpy-Arrays
Das Python-Package scikit-learn setzt für die Verarbeitung auf numpy-Arrays auf, aus dem diesem Grund werden die pandas-Dataframes in numpy-arrays konvertiert.

In [7]:
# Umwandlung in numpy-Arrays

# Trainingsdaten als numpy-Array
#train_recoded_num = data_train.apply(LabelEncoder().fit_transform).to_numpy() # recoded die nominalen Merkmale in Zahlen
#x_train_recoded_num = np.array(train_recoded_num[:, :4]) # die ersten 4 Spalten inkl. aller Spalten werden aus der Matrix geschnitten
#y_train_recoded_num = np.array(train_recoded_num[:, 4]) # schneidet aus der Matrixe die Spalte mit den Zielwerten (= Outcome,..) 

# Testdaten als numpy-Array
#test_recoded_num = data_test.apply(LabelEncoder().fit_transform).to_numpy() # recoded die nominalen Merkmale in Zahlen
#x_test_recoded_num = np.array(test_recoded_num[:, :4]) # die ersten 4 Spalten inkl. aller Spalten werden aus der Matrix geschnitten
#y_test_recoded_num = np.array(test_recoded_num[:, 4]) # schneidet aus der Matrixe die Spalte mit den Zielwerten (= Outcome,..) 

# Bezeichner für Features
#class_names = np.unique(y_train_recoded_num) # schreibt die Labels als array. Labels sind die Bezeichner der Klassenzugehörigkeit 

# Bezeichner für Labels wird auf "no" und "yes" umcodiert
#class_names = np.where(class_names == 0, "no", "yes") # die Labels als Text, wichtig später für den Tree 

## Aufstellen des Modells  


Wir möchten nun den decision tree erstellen, der eine Klassifikation durchführen kann. 
Wir verwenden dafür im Folgenden das weit verbreitete `scikit-learn` als spezialisiertes Python-Package für Aufgaben aus dem Data-Science-Umfeld. 
Für die Erstellung des Modells müssen wir uns auf einen Ansatz bzw. Kriterium festlegen, um die Knoten zu finden, bei denen sich die Entscheidung gabelt. 
Bei `scikit-learn` lässt sich einstellen, welches Kriterium als Diskriminanzmaß zu verwenden ist (z.B. Gini-Koeffizient, Entropie usw.) 
Wir nutzen im Folgenden als Kriterium für das Finden der Knoten die "Entropie". 

*Aufgabe*: Erstellen Sie eine Instanz des Modells und setzten Sie als Kriterium die "Entropie" und als maximale Tiefe des Baums "3". 

In [ ]:
from sklearn.tree import DecisionTreeClassifier


*Aufgabe*: Fitten bzw. Trainieren Sie das Modell mit den Trainingsdaten

## Visualisierung als Decision Tree
Für die Visualisierung des Entscheidungsbaums nutzen wir zunächst matplotlib. Eine weitere Visualisierungsmöglichkeit ist [Graphviz](https://www.graphviz.org/) - ein Werkzeug speziell für die Darstellung von Graphen. 

*Aufgabe*: Visualisieren Sie den Entscheidungsbaum über matplotlip:

In [ ]:
from sklearn import tree
fig, axes = mpl.subplots(nrows = 1,ncols = 1,figsize = (3,4), dpi=270)


Der erste Knoten vom Entscheidungsbaum legt die Entscheidung nahe, Tennis zu spielen, wenn das Wetter bevölkt ist ("overcast" <= 0, durch die vorherige Recodierung wurde "overcast" mit "0" annotiert). 

## Interaktive Visualisierung (Optional)
Mit wenig Code sind auch interaktive Diagramme möglich.
Interaktive Diagramme sind nützlich, wenn die Wahl der Parameter  eine hohe Auswirkung auf das Ergebnis haben kann - wie etwa bei "Decision Tree".

Für interaktive Charts muss das Python-Package `ipwidgets`installiert sein, welches Sie vorab via `conda install -c conda-forge ipywidgets` in der Anaconda-Console installieren müssen.

*Aufgabe*: Versuchen Sie eine interaktive Visualisierung mit dem Python-Package ipwidgets zu erstellen.

In [2]:
# Voraussetzung "conda install -c conda-forge ipywidgets" in der Anaconda Console ausgeführt

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display                               
from ipywidgets import interactive

**Exkurs:**
Berechnen der Entropie - dadurch wird auch der Startknoten festgelegt, denn das Merkmal mit der höchsten Entropie bildet die Wurzel ("root") beim Entscheidungsbaum. Das Konstruktionsprinzip sieht vor, mit dem Merkmal zu beginnen, das die höchste Entropie aufweist, dann fortzusetzen mit dem nächsten Merkmal, bei dem die Entropie dann am höchsten ist usw.   

**Exkurs:** Reihung der Features nach ihrem Beitrag zum Modell. Das Feature "Outlook" hat das größte Gewicht auf die Entscheidung, d.h. die Merkmalsausprägungen von "Outlook" wie "overcast", "sunny" und "rain" bedingen eine Entscheidung am stärksten.

## Testen des Modells

Nachdem wir das Modell trainiert haben, testen wir das trainierte Modell! 
Die Labels der Testdaten sind bekannt, die Evaluation besteht aber darin, zunächst die Labels dem Classifier nicht zu "verraten" und das Modell so anzuwenden, als sei die Information nach der Klassenzugehörigkeit nicht bekannt.

*Aufgabe*: Testen Sie ihr trainiertes Modell. Verwenden Sie dazu am besten Datensatz "x_test_recoded_num".
Gehen Sie dabei am besten wie folgt vor: 
1. Y-Variable (Output-Variable) mittels dem zuvor trainierten Modell für die Testdaten ermitteln
2. Gegenüberstellung der vorhergesagten und den "richtigen" Werten (am besten in einer Tabelle)

# Evaluation 

## Confusion-Matrix
Ein gängiges Verfahren zur Evaluation der Klassifikationsergebnisse ist das Aufstellen einer Confusion-Matrix als Grundlage für weiterführende Vergleiche auf Grundlage von Kennzahlen wie "Recall" oder "Precision".


*Aufgabe*: Erstellen sie eine Convusion-Matrix. 
Hinweis: Dafür müssen sie zuerst die Häufigkeiten für die Confusion-Matrix bestimmen mit der Funktion "metrics_confusion_matrix". 

In [ ]:
from sklearn import metrics


### Accuracy, Recall und Precision

*Aufgabe*: Berechnen Sie aus der Confusion-Matrix noch die Kennzahlen zur Beurteilung der Modellgüte und interpretieren Sie die Ergebnisse: 
- Accuracy = (TP + TN) / N -> metrics.accuracy_score
- Recall = TP / (TP + FN) -> metrics.precision_score
- Precision = TP / (TP + FP) -> metrics.recall_score

## ROC-Kurve (Optional)
Die ROC (Receiver Operating Characteristics) ist ein weiteres Kriterium zur Beurteilung der Modellgüte. Die ROC-Kurve ist eine Grafik, bei der die TP- der FP-Rate gegenübergestellt wird. Idealtypisch verliefe die Kurve bei einem perfekten Modell sofort bis zur 1 und ab dann waagrecht, die ROC-Area läge dann auch (idealtypisch) bei 1. Ein minderwertiges Modell entlarvt eine ROC-Kurve, wenn sie entlang oder gar unterhalb der 45°-Diagonalen verliefe. Beachten Sie, dass Raten auch eine Chance von 50:50 auf das richtige Ergebnis hätte. Ein Modell wird umso besser, je weiter es oberhalb der roten 45°-Diagonale verläuft. 

*Aufgabe*: Berechnen Sie zuerst für die ROC-Kurve die TP- und FP-Rate auf Grundlage der Confusion-Matrix.

In [ ]:
from sklearn.metrics import confusion_matrix


*Aufgabe*: Erstellen Sie nun die ROC-Kurve

In [ ]:
from sklearn.metrics import roc_curve, auc
